# Generic HMC 

### Imports

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from l2hmc_eager import dynamics_eager as _l2hmc
from l2hmc_eager import gauge_dynamics_eager as l2hmc
from l2hmc_eager.neural_nets import *
from utils.distributions import GMM, gen_ring
from utils.jacobian import _map, jacobian
from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)

from HMC.hmc import HMC

from lattice.gauge_lattice import GaugeLattice, pbc, mat_adj, u1_plaq_exact

%autoreload 2

Using TensorFlow backend.


In [18]:
from u1_model_eager import *

In [3]:
tf.enable_eager_execution()
tfe = tf.contrib.eager

In [4]:
def train_one_iter(dynamics, x, optimizer, loss_fn=l2hmc.compute_loss, 
                   scale=0.1, eps=1e-4, l2_dist=True, global_step=None):
    loss, grads, out, accept_prob = l2hmc.loss_and_grads(
        dynamics, x, loss_fn=loss_fn, scale=scale, eps=eps, l2_dist=l2_dist
    )
    gradients, _ = tf.clip_by_global_norm(grads, 10.)
    optimizer.apply_gradients(
        zip(grads, dynamics.trainable_variables), global_step=global_step
    )
    return loss, out, accept_prob

In [5]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

In [6]:
def compute_ac_spectrum(samples_history, target_mean, target_covar):
    """Compute autocorrelation spectrum.
    Follows equation 15 from the L2HMC paper.
    Args:
        samples_history: Numpy array of shape [T, B, D], where T is the total
            number of time steps, B is the batch size, and D is the dimensionality
            of sample space.
        target_mean: 1D Numpy array of the mean of target(true) distribution.
        target_covar: 2D Numpy array representing a symmetric matrix for 
            variance.
    Returns:
        Autocorrelation spectrum, Numpy array of shape [T-1].
    """
    # Using numpy here since eager is a bit slow due to the loop
    time_steps = samples_history.shape[0]
    #trace = np.trace(target_covar)
    trace = 1.
    rhos = []
    for t in range(time_steps - 1):
        rho_t = 0.
        for tau in range(time_steps - t):
            v_tau = samples_history[tau, :] - target_mean
            v_tau_plus_t = samples_history[tau + t, :] - target_mean
            # Take dot product over observation dims and take mean over batch dims
            rho_t = v_tau.T.dot(v_tau_plus_t)
            #rho_t += np.mean(np.sum(v_tau * v_tau_plus_t, axis=1))
        rho_t /= trace * (time_steps - t)
        rhos.append(rho_t)
    return np.array(rhos)


## 2D $U(1)$ Lattice Gauge Theory

### Using L2HMC framework with hmc flag. `hmc=True`

In [23]:
params = {
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'beta': 8.,
    'num_samples': 2,
    'num_steps': 5,
    'eps': 0.05,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 1e-4,
    'learning_rate_decay_steps': 100,
    'learning_rate_decay_rate': 0.96,
    'train_steps': 1000,
    'record_loss_every': 50,
    'data_steps': 10,
    'save_steps': 50,
    'print_steps': 1,
    'logging_steps': 5,
    'clip_value': 100,
    'rand': False,
    'metric': 'l2',
    #'conv_net': False,
    #'hmc': True,
}
tf.reset_default_graph()

In [24]:
model_hmc = GaugeModelEager(params=params,
                            conv_net=False,
                            hmc=True,
                            log_dir=None,
                            restore=False,
                            defun=True)

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_eager/run_128/
Creating lattice...
done.
Creating dynamics...
done.
total initialization time: 0.029989242553710938

################################################################################
Model parameters:
log_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_eager/run_128/

info_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_eager/run_128/run_info/

figs_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_eager/run_128/figures/

_defun: True

conv_net: False

hmc: True

time_size: 8

space_size: 8

link_type: U1

dim: 2

beta: 8.0

num_samples: 2

num_steps: 5

eps: 0.05

loss_scale: 0.1

loss_eps: 0.0001

learning_rate_init: 0.0001

learning_rate_decay_steps: 100

learning_rate_decay_rate: 0.96

train_steps: 1000

record_loss_every: 50

data_steps: 10

save_steps: 50

print_steps: 1

logging_steps: 5

clip_value: 100

rand: False

metric: l2

batch_size: 2

########################################################################

In [25]:
observables_hmc = model_hmc.calc_observables(model_hmc.samples, 
                                             _print=True, 
                                             _write=True)
total_actions, avg_plaquettes, top_charges = observables_hmc

    STEP        LOSS   TIME/STEP  ACCEPT %    EPS      ACTION    TOP Q      PLAQ   
------------------------------------------------------------------------------------
    0/1000       0         0         0        0.05       0         0         1     




In [19]:
start_step = model_hmc.global_step.numpy()
samples = model_hmc.samples

loss, samples, accept_prob, grads = train_one_iter(
    dynamics=model_hmc.dynamics,
    samples=samples,
    optimizer=model_hmc.optimizer,
    loss_fn=model_hmc.loss_fn,
    global_step=model_hmc.global_step,
    params=model_hmc.params,
    hmc=model_hmc.hmc
)

In [26]:
model_hmc.train(500, keep_samples=False)

KeyboardInterrupt: 

### Using HMC.hmc method (separate from L2HMC)

In [19]:
lattice = GaugeLattice(time_size=params['time_size'],
                       space_size=params['space_size'],
                       dim=params['dim'],
                       beta=params['beta'],
                       link_type=params['link_type'],
                       num_samples=params['num_samples'],
                       rand=params['rand'])
#samples = np.array([sample.flatten() for sample in lattice.samples])
lattice_energy_fn = lattice.get_energy_function()

In [20]:
#step_size = params['eps']
step_size = 0.05
#n_leapfrog_steps = params['n_steps']
n_leapfrog_steps = 5
#position_init = samples
position_init = lattice.links.flatten()
lattice_hmc = HMC(position_init=position_init,
                  step_size=step_size,
                  n_leapfrog_steps=n_leapfrog_steps,
                  potential_fn=lattice_energy_fn,
                  beta=lattice.beta)

In [21]:
# 500 steps in ~ 6m 41s
links0 = lattice.links.flatten()
momentum0 = np.random.randn(*links0.shape)
links_arr = [links0]
vel_arr = []
probs_arr = []
total_actions = []
average_plaquettes = []
topological_charges = []
samples_arr = []
links1 = links0
num_steps = 500
print("Exact value of the average plaquette "
      f"(at {params['beta']}): {u1_plaq_exact(params['beta'])}\n")

Exact value of the average plaquette (at 8.0): 0.9352354935294382



In [22]:
for i in range(num_steps):
    t1 = time.time()
    if isinstance(links1, tf.Tensor):
        samples_arr.append(links1.numpy())
    else: 
        samples_arr.append(links1)
    links1, vel1, probs1 = lattice_hmc.apply_transition(links1)
    
    observables = np.array(lattice._calc_plaq_observables(links1))
    _total_actions = observables[0]
    _avg_plaquettes = observables[1]
    _top_charges = observables[2]
    
    total_actions.append(_total_actions)
    average_plaquettes.append(_avg_plaquettes)
    topological_charges.append(_top_charges)
    print(f'\nstep: {i:<5g} accept rate: {np.mean(probs1):^8.5g}  '
      f' time/step: {time.time() - t1:^6.4g} '
      f' avg_S: {np.mean(_total_actions):^8.5g} '
      f' avg_topQ: {np.mean(_top_charges):^8.5g} '
      f' avg_plaq: {np.mean(_avg_plaquettes):^8.5g}\n ')


step: 0     accept rate: 0.46315    time/step: 0.9393  avg_S:  35.401   avg_topQ: 0.099018  avg_plaq: 0.93086 
 

step: 1     accept rate:    1       time/step: 0.9632  avg_S:  32.466   avg_topQ: 0.031778  avg_plaq: 0.93659 
 

step: 2     accept rate:    1       time/step: 0.9292  avg_S:  24.82    avg_topQ: 0.035451  avg_plaq: 0.95152 
 

step: 3     accept rate: 0.76233    time/step: 1.056   avg_S:  35.84    avg_topQ: -0.050784  avg_plaq:   0.93  
 

step: 4     accept rate: 0.82029    time/step: 0.961   avg_S:  49.175   avg_topQ: 0.085133  avg_plaq: 0.90396 
 

step: 5     accept rate:    1       time/step: 1.017   avg_S:  35.858   avg_topQ: 0.026927  avg_plaq: 0.92997 
 

step: 6     accept rate:    1       time/step: 1.042   avg_S:  33.347   avg_topQ: 0.061325  avg_plaq: 0.93487 
 

step: 7     accept rate:    1       time/step: 1.023   avg_S:  22.991   avg_topQ: 0.046442  avg_plaq:  0.9551 
 

step: 8     accept rate: 0.83292    time/step: 0.9618  avg_S:  29.822   avg_topQ: -0.0

KeyboardInterrupt: 

In [ ]:
_samples_arr = []
for i in range(len(samples_arr)):
    if isinstance(samples_arr[i], tf.Tensor):
        print(f'{i} (is tensor)')
        _samples_arr.append(samples_arr[i].numpy())
    else:
        print(f'{i} (is not tensor)')
        _samples_arr.append(samples_arr[i])

In [ ]:
_samples_arr = np.array(_samples_arr)
_samples_arr.shape

In [ ]:
target_mean = np.mean(_samples_arr, axis=0)

In [ ]:
samples_autocorr = compute_ac_spectrum(_samples_arr, target_mean=target_mean,
                                       target_covar=None)

In [ ]:
#apply_transition = tfe.defun(lattice_hmc.apply_transition)

#_samples = tf.random_normal(shape=links1.shape)
#_samples = np.random.randn(*links1.shape)
#samples_arr = []
#actions_arr = []
#plaquettes_arr = []
#top_charges_arr = []
#for i in range(100):
    #samples_arr.append(_samples)
    #_samples, _, _ = apply_transition(_samples)
    #
    #observables = np.array(lattice._calc_plaq_observables(_samples))
    #_total_actions = observables[0]
    #_avg_plaquettes = observables[1]
    #_top_charges = observables[2]
    #
    #actions_arr.append(_total_actions)
    #plaquettes_arr.append(_avg_plaquettes)
    #top_charges_arr.append(_top_charges)

In [ ]:
samples_arr[0]

In [ ]:
samples_arr[-1]

In [ ]:
_samples_arr = [samples_arr[0]]

In [ ]:
type(samples_arr[1])

In [ ]:
isinstance(samples_arr[1], tf.Tensor)

In [ ]:
samples_arr[2].numpy()

In [ ]:
_samples_arr.append([i.numpy() for i in samples_arr if isinstance(i, tf.Tensor)])

In [ ]:
_samples_arr = np.array(_samples_arr)
_samples_arr.shape

In [ ]:
samples_arr = np.array([sample.numpy() for sample in samples_arr])
samples_arr.shape

In [ ]:
samples_arr[1]

In [ ]:
samples_autocorr = autocorr(samples_arr)

In [ ]:
top_charge_autocorr4 = autocorr(np.array(topological_charges))
steps = np.arange(len(top_charge_autocorr))

In [ ]:
top_charge_autocorr3 = autocorr(np.array(topological_charges))
steps = np.arange(len(top_charge_autocorr))

In [ ]:
top_charge_autocorr2 = autocorr(np.array(topological_charges))
steps = np.arange(len(top_charge_autocorr))

In [ ]:
top_charge_autocorr1 = autocorr(np.array(topological_charges))
steps = np.arange(len(top_charge_autocorr))

In [ ]:
top_charge_autocorr = autocorr(np.array(topological_charges))
steps = np.arange(len(top_charge_autocorr))

In [ ]:
samples_autocorr = autocorr(np.array())

## Create plots

In [ ]:
%matplotlib notebook

In [ ]:
steps = np.arange(len(top_charge_autocorr))
fig, ax = plt.subplots()
ax.plot(steps, top_charge_autocorr, ls='-', marker='', 
        label=f"HMC (eps: {params['eps']}, 5 steps)")
ax.plot(steps, top_charge_autocorr1, ls='-', marker='', 
        label=f"HMC (eps: {0.1}, 5 steps)")
ax.plot(steps, top_charge_autocorr2, ls='-', marker='', 
        label=f"HMC (eps: {0.025}, 5 steps)")
ax.plot(steps, top_charge_autocorr, ls='-', marker='', 
        label=f"HMC (eps: {0.05}, 10 steps)")
ax.plot(steps, top_charge_autocorr4, ls='-', marker='', 
        label=f"HMC (eps: {0.025}, 10 steps)")
ax.set_xlabel('Gradient computations')
ax.set_ylabel('Autocorrelation of Topological Charge')
#ax.set_xlim((-2, 50))
ax.legend(loc='best')
fig.savefig(
    '../../figures/HMC_autocorrelation_fn/top_charge_autocorr_no_l2hmc.pdf', 
    dpi=400, bbox_inches='tight'
)
plt.show()

In [ ]:
steps = np.arange(len(samples_autocorr))
fig, ax = plt.subplots()
ax.semilogy(steps, samples_autocorr, ls='-', marker='', 
        label=f"HMC (eps: {params['eps']}, 5 steps)")
ax.set_xlabel('Gradient computations')
ax.set_ylabel('Autocorrelation from Samples')
#ax.set_xlim((-2, 50))
ax.legend(loc='best')
#fig.savefig(
#    '../../figures/HMC_autocorrelation_fn/top_charge_autocorr_no_l2hmc.pdf', 
#    dpi=400, bbox_inches='tight'
#)
plt.show()

In [ ]:
np.mean(average_plaquettes)

In [ ]:
print(u1_plaq_exact(beta))

## Using L2HMC with auxiliary functions $Q, S, T \equiv 0$ (i.e. generic HMC)

In [ ]:
##########################  Parameters  #####################################
# n_steps: number of leapfrog steps, eps: initial step size for dynamics
# loss_scale: scaling factor (lambda^2 in paper) in loss objective
# loss_eps: for numeric stability in loss function
# beta: inverse coupling strength
##############################################################################
time_size, space_size, dim, beta, num_samples = (4, 4, 2, 3., 4)
n_steps, eps, loss_scale, loss_eps = (10, 0.1, .1, 1e-4)
rand=True
l2_dist = True
conv_net = True

In [ ]:
u1_lattice = GaugeLattice(time_size, space_size, dim, beta,
                          link_type='U1', num_samples=num_samples, rand=rand)
if conv_net:
    u1_samples_tensor = tf.convert_to_tensor(u1_lattice.samples, 
                                             dtype=tf.float32)
else:
    flat_samples = [sample.flatten() for sample in u1_lattice.samples]
    u1_samples_tensor = tf.convert_to_tensor(np.stack(flat_samples), 
                                             dtype=tf.float32)

# Construct dynamics object
u1_energy_fn = u1_lattice.get_energy_function(u1_samples_tensor)
u1_dynamics = l2hmc.GaugeDynamics(u1_lattice, n_steps=n_steps, eps=eps,
                                  minus_loglikelihood_fn=u1_energy_fn, 
                                  conv_net=conv_net, test_HMC=True)

In [ ]:
global_step = tf.train.get_or_create_global_step()
_ = global_step.assign(1)
train_iters = 500
record_loss_every = 50
save_steps = 50 

learning_rate = tf.train.exponential_decay(1e-2, global_step, 50,
                                           0.96, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)
checkpointer = tf.train.Checkpoint(
    optimizer=optimizer, dynamics=u1_dynamics, global_step=global_step
)
#summary_writer = tf.contrib.summary.create_file_writer(log_dir)
loss_fn = l2hmc.compute_loss

print(u1_plaq_exact(beta))

In [ ]:
#################    Run L2HMC algorithm    ##################################
total_actions = []
average_plaquettes = []
topological_charges = []
samples = u1_samples_tensor

In [ ]:
t0 = time.time()
start_step = global_step.numpy()
for i in range(start_step, 1000):
    t1 = time.time()
    loss, samples, accept_prob = train_one_iter(
        u1_dynamics,
        samples,
        optimizer,
        loss_fn=loss_fn,
        scale=loss_scale,
        eps=loss_eps,
        global_step=global_step
    )
    observables = np.array(u1_lattice.calc_plaq_observables(samples))
    _total_actions = observables[:, 0]
    _avg_plaquettes = observables[:, 1]
    _top_charges = observables[:, 2]
    
    total_actions.append(_total_actions)
    average_plaquettes.append(_avg_plaquettes)
    topological_charges.append(_top_charges)
    
    print(f'\nstep: {i:<5g} loss: {loss.numpy():^8.5g} '
          f' time/step: {time.time() - t1:^6.4g} '
          f' accept: {accept_prob.numpy().mean():^8.5g} '
          f' eps: {u1_dynamics.eps.numpy():^6.4g} '
          f' avg_S: {np.mean(_total_actions):^8.5g} '
          f' avg_topQ: {np.mean(_top_charges):^8.5g} '
          f' avg_plaq: {np.mean(_avg_plaquettes):^8.5g}\n ')
    print('avg_plaquettes: {}\n'.format([_avg_plaquettes]))

In [ ]:
samples = u1_samples_tensor
print(samples.shape)

In [ ]:
x = tf.reshape(samples, shape=[samples.shape[0], -1])
y = tf.random_normal(x.shape)

In [ ]:
xy = tf.matmul(x, y, transpose_b=True)

In [ ]:
xy_loss = tf.reduce_sum(xy / (tf.norm(x) * tf.norm(y)), axis=1)

In [ ]:
loss = tf.reduce_mean((loss_scale / xy_loss - xy_loss / loss_scale), axis=0)

In [ ]:
loss

In [ ]:
help(tf.clip_by_global_norm)

In [ ]:
tf.abs

In [ ]:
ss = tf.matmul(samples, samples)
print(ss.shape)

In [ ]:
avg_plaqs_arr = np.array(average_plaquettes)
_avg_plaqs_arr = np.mean(avg_plaqs_arr, axis=0)
avg_plaq, avg_plaq_err = calc_avg_vals_errors(avg_plaqs_arr[450:500], num_blocks=50)
print(f'avg_plaq (mean from arr): {np.mean(_avg_plaqs_arr)}')
print(f'avg_plaq: {avg_plaq} +/- {avg_plaq_err}')

In [ ]:
np.mean(average_plaquettes[-100:])

In [ ]:
def project_angle(x):
    """Function to project an angle from [-4pi, 4pi] to [-pi, pi]."""
    return x - 2 * np.pi * tf.math.floor((x + np.pi) / (2 * np.pi))

In [ ]:
project_angle(-2 * np.pi)

In [ ]:
t = np.arange(-10, 10, 0.05)
y = project_angle(t)

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, y, 'o')

## Strongly Correlated Gaussian target distribution (for testing HMC implementation)

### Define log density function of target distribution (potential energy function) $S(x)$

$$ S(\mathbf{x}) = \frac{-\frac{1}{2} (\mathbf{x} - \mathbf{\mu})^{T} \mathbf{\Sigma}^{-1}(\mathbf{x} - \mathbf{\mu})}{\sqrt{|\Sigma|}} $$ 


In [ ]:
mu = np.zeros(2)
cov = np.array([[1., 0.95], [0.95, 1.]], dtype=np.float32)
cov_inv = np.linalg.inv(np.copy(cov))
def quadratic_gaussian(x):
    x_mu = x - mu
    return 0.5 * tf.reduce_sum(tf.transpose(x_mu) * cov_inv * x_mu)

def quadratic_gaussian_grad(x):
    return x

### Exact target distribution

In [ ]:
mean = [0, 0]
cov = [[1., 0.95], [0.95, 1.]]
samples_x, samples_y = np.random.multivariate_normal(mean, cov, 1000).T

### Instantiate HMC object for sampling

In [ ]:
step_size = 0.1
n_leapfrog_steps = 15
hmc = HMC(position_init=np.random.randn(2), 
          step_size=step_size,
          n_leapfrog_steps=n_leapfrog_steps,
          potential_fn=quadratic_gaussian,
          grad_potential_fn=grad_quad_gaussian)
          #grad_potential_fn=grad_quad_gaussian)

In [ ]:
pos0 = [[-1., 1.]]
pos = [pos0]
vel = []
probs = []
pos1 = pos0

for i in range(500):
    #pos0 = pos[i-1]
    pos1, vel1, probs1 = hmc.apply_transition(pos1)
    pos.append(pos1)
    vel.append(vel1)
    probs.append(probs1)
pos = np.array(pos).reshape(len(pos), -1)
vel = np.array(vel)
probs = np.array(probs)

In [ ]:
fig, ax = plt.subplots()
_ = ax.plot(samples_x, samples_y, marker='o', ls='', alpha=0.45)
_ = ax.plot(pos[:,0], pos[:,1], marker='o', ls='', alpha=0.6)
_ = ax.set_title('500 transitions, numerical gradient')
plt.show()

### Look at the leapfrog integrator to tune hyperparameters

In [ ]:
x0 = np.array([-1., 1.])
p0 = np.random.randn(*np.array(x0).shape)
x, p = x0, p0
x_arr = []
p_arr = []
for i in range(n_leapfrog_steps):
    lf_out = hmc._leapfrog_fn(x, p, i)
    x, p = lf_out
    x_arr.append(x)
    p_arr.append(p)
x_arr = np.array(x_arr)
p_arr = np.array(p_arr)

In [ ]:
fig, ax = plt.subplots()
_ = ax.plot(samples_x, samples_y, marker='o', ls='', alpha=0.45)
_ = ax.plot(x_arr[:,0], x_arr[:,1], marker='.', ls='-', alpha=0.6)
plt.show()

In [ ]:
fig, ax = plt.subplots()
_ = ax.plot(samples_x, samples_y, marker='o', ls='', alpha=0.45)
_ = ax.plot(pos[:,0], pos[:,1], marker='o', ls='', alpha=0.6)
_ = ax.set_title('500 transitions, numerical gradient')
plt.show()